In [26]:
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
housing = fetch_california_housing()
import tensorflow as tf

In [27]:
X_train_full ,X_test,y_train_full,y_test = train_test_split(
    housing.data,housing.target
)
X_train,X_valid,y_train,y_valid = train_test_split(
    X_train_full,y_train_full
)

In [4]:
class WideAndDeepModel(keras.Model):
  def __init__(self,units=30,activation="relu",**kwargs):
    super().__init__(**kwargs)
    self.hidden1 = keras.layers.Dense(units,activation=activation)
    self.hidden2 = keras.layers.Dense(units,activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)

  def call(self,inputs):
    input_A,input_B = inputs
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A,hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output,aux_output
  
model = WideAndDeepModel()


In [14]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")

X_train_A,X_train_B, = X_train[:,:5],X_train[:,2:]
X_valid_A,X_valid_B = X_valid[:,:5],X_valid[:,2:]
X_test_A,X_test_B = X_test[:,:5],X_test[:,2:]
X_new_A,X_new_B = X_test_A[:3],X_test_B[:3]

In [ ]:
history = model.fit(
    [X_train_A,X_train_B],[y_train,y_train],epochs=20,
    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid])
)
total_loss,main_loss,aux_loss = model.evaluate(
    [X_test_A,X_test_B],[y_test,y_test]
)
y_pred_main,y_pred_aux = model.predict([X_new_A,X_new_B])

Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan - val_loss: nan - val_output_1_loss: nan - val_output_2_loss: nan
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan - val_loss: nan - val_output_1_loss: nan - val_output_2_loss: nan
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan - val_loss: nan - val_output_1_loss: nan - val_output_2_loss: nan
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan - val_loss: nan - val_output_1_loss: nan - val_output_2_loss: nan
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: nan - output_1_loss: nan - output_2_loss: nan - val_loss: nan - val_output_1_loss: nan - val_output_2_loss: nan
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - lo

In [ ]:
import os
root_logdir=os.path.join(os.curdir,"my_logs")
root_logdir

'./my_logs'

In [ ]:
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%_d-%H_%M_%S")
  return os.path.join(root_logdir,run_id)

In [ ]:
run_logdir = get_run_logdir()

In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit( [X_train_A,X_train_B],[y_train,y_train],epochs=30,validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]),
                    callbacks=[tensorboard_cb])

# 调整神经网络参数

In [21]:
def build_model(n_hidden=1,n_neurons=30,learning_rate=3e-3,input_shape=[8]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,activation="relu"))
  model.add(keras.layers.Dense(1))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss="mse",optimizer=optimizer)
  return model


In [22]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

<ipython-input-22-651c14c6d32f>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [23]:
keras_reg.fit(X_train,y_train,epochs=100,
              validation_data=(X_valid,y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 8/100
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 9/100
363/363 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: nan - val_loss: nan


In [28]:
mse_test = keras_reg.score(X_test,y_test)
mse_test

162/162 [==============================] - 0s 1ms/step - loss: nan


nan

In [17]:
y_pred = keras_reg.predict(X_test)
y_pred

162/162 [==============================] - 1s 4ms/step


array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)

In [30]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
param_distribs = {
    "n_hidden":[0,1,2,3],
    "n_neurons":np.arange(1,100),
    "learning_rate":reciprocal(3e-4,3e-2),

}


In [31]:
rnd_search_cv = RandomizedSearchCV(keras_reg,param_distribs,n_iter=10,cv=3)

In [ ]:
rnd_search_cv.fit(X_train,y_train,epochs=100,
              validation_data=(X_valid,y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [33]:
rnd_search_cv.best_params_

{'learning_rate': 0.00031078262569536066, 'n_hidden': 2, 'n_neurons': 69}

In [34]:
rnd_search_cv.best_score_

nan